In [2]:
from langgraph.graph import StateGraph, START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict,Literal
from pydantic import BaseModel,Field

In [3]:
load_dotenv()
llm_model = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.7
)

In [4]:
class SentimentSchema(BaseModel):
    sentiment : Literal["positive","negative"] = Field(description="Sentiment of the review")
    
class DiagnosisSchema(BaseModel):
    issue_type : Literal['UX','Performance','Bug','Support','Other'] = Field(description="Yhe category of the issue mentioned in the review")
    tone : Literal['Angry','Frustrated','Disappointed','calm'] = Field(description='Yhe emotional toned express by the user')
    urgency : Literal["low",'medium','high'] = Field(description="How urgent or ctitical the issue appears to be")

In [5]:
structured_model = llm_model.with_structured_output(SentimentSchema)
structured_model2 = llm_model.with_structured_output(DiagnosisSchema)

In [6]:
prompt = "What is the sentiment of the following review - The software too good"
structured_model.invoke(prompt).sentiment

'positive'

In [7]:
## Define State

class ReviewState(TypedDict):
    review : str
    sentiment : Literal["positive","negative"]
    diagnosis : dict
    response : str

In [25]:
def find_sentiment(state : ReviewState):
    
    prompt = f"For the following review find out the sentiment \n{state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    return {"sentiment": sentiment}


def check_sentiment(state : ReviewState) -> Literal["positive_response","run_diagnosis"]:
    if state["sentiment"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"
    
def positive_response(state:ReviewState):
    prompt = f""" Write a warm thank-you message in response to this review :
    \n\n"{state['review']}\n"\n
    Also kindly ask the user to leave feedback on our website."""
    
    
    response = llm_model.invoke(prompt).content
    return {"response":response }

def run_diagnosis(state:ReviewState):
    prompt = f'Diagnose this negative review : \n\n{state['review']}\n'
    
    response = structured_model2.invoke(prompt)
    return{'diagnosis': response.model_dump()}
    

def negative_response(state:ReviewState):
    diagnosis = state['diagnosis']
    prompt = f""" You are a support assistant.
    The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}',
    Write an empathetic , helpful resulution message.
    """
    
    response = llm_model.invoke(prompt).content
    return {'response': response}

In [26]:
# define graph

graph = StateGraph(ReviewState)
graph.add_node("find_sentiment",find_sentiment)
graph.add_node("positive_response",positive_response)
graph.add_node("run_diagnosis",run_diagnosis)
graph.add_node("negative_response",negative_response)

## add edges
graph.add_edge(START,'find_sentiment')
graph.add_conditional_edges('find_sentiment',check_sentiment)
graph.add_edge('positive_response',END)
graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response',END)

workflow = graph.compile()



In [27]:
initial_state = {
    'review': "Disappointed with the product. It did not perform as expected, the quality feels average, and it is not worth the price. There are better alternatives available."
}

workflow.invoke(initial_state)

{'review': 'Disappointed with the product. It did not perform as expected, the quality feels average, and it is not worth the price. There are better alternatives available.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Other',
  'tone': 'Disappointed',
  'urgency': 'medium'},
 'response': "I'm genuinely sorry to hear that you're feeling disappointed. It's completely understandable to feel that way when things aren't working as expected or when you encounter an unexpected issue. Please know that even though your issue is categorized as 'Other', it's no less important, and my top priority is to help you.\n\nI see this is marked with medium urgency, and I assure you I'll be giving it my full attention to find a helpful solution as quickly as possible.\n\nTo help me understand exactly what happened and how it's impacting you, could you please provide a few more details? For example:\n\n*   What specifically occurred?\n*   When did you first notice the problem?\n*   Are there a